In [1]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
import pandas as pd
from sklearn.utils import shuffle, resample
from afqinsight.augmentation import jitter, time_warp, scaling
import tempfile

In [2]:
afq_dataset = AFQDataset.from_files(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md", "dki_mk"],
    index_col="subject_id",
    target_cols=["age", "dl_qc_score", "scan_site_id"],
    label_encode_cols=["scan_site_id"]
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1865
(1865, 7200)
(1865, 3)


In [5]:
full_dataset = list(afq_dataset.as_tensorflow_dataset().as_numpy_iterator())

2022-05-06 10:56:00.435244: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
X = np.concatenate([xx[0][None] for xx in full_dataset], 0)
y = np.array([yy[1][0] for yy in full_dataset])
qc = np.array([yy[1][1] for yy in full_dataset])
site = np.array([yy[1][2] for yy in full_dataset])

In [7]:
X = X[qc>0]
y = y[qc>0]
site = site[qc>0]

In [8]:
n_epochs = 1000

# EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    mode="min",
    patience=100
)

# ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=20,
    verbose=1,
)

In [9]:
def augment_this(X, y, rounds=2): 
    new_X = X[:]
    new_y = y[:]
    for f in range(rounds): 
        aug_X = np.zeros_like(X)
        # Do each channel separately:
        for channel in range(aug_X.shape[-1]):
            this_X = X[..., channel][..., np.newaxis]
            this_X = jitter(this_X, sigma=np.mean(this_X)/25)
            this_X = scaling(this_X, sigma=np.mean(this_X)/25)
            this_X = time_warp(this_X, sigma=np.mean(this_X)/25)
            aug_X[..., channel] = this_X[...,0]
        new_X = np.concatenate([new_X, aug_X])
        new_y = np.concatenate([new_y, y])
    return new_X, new_y 

In [10]:
# Generate evaluation results and correlation coeffcients combined in a dataframe, and history
def single_cross_site(model_name, name_str, lr,
                      site_1, site_2, site_3, X, y):
    # Split the data by sites
    X_1 = X[site==site_1]
    y_1 = y[site==site_1]
    X_2 = X[site==site_2]
    y_2 = y[site==site_2]
    X_3 = X[site==site_3]
    y_3 = y[site==site_3]
    # Split the data into train and test sets:
    X_train, X_test1, y_train, y_test1 = train_test_split(X_1, y_1, test_size=0.2)
    _, X_test2, _, y_test2 = train_test_split(X_2, y_2, test_size=0.2)
    _, X_test3, _, y_test3 = train_test_split(X_3, y_3, test_size=0.2)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test1 = np.concatenate([imputer.fit_transform(X_test1[..., ii])[:, :, None] for ii in range(X_test1.shape[-1])], -1)
    X_test2 = np.concatenate([imputer.fit_transform(X_test2[..., ii])[:, :, None] for ii in range(X_test2.shape[-1])], -1)
    X_test3 = np.concatenate([imputer.fit_transform(X_test3[..., ii])[:, :, None] for ii in range(X_test3.shape[-1])], -1)
    model = model_name(input_shape=(100, 72), n_classes=1, output_activation=None, verbose=True)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['mean_squared_error', 
                           tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                           'mean_absolute_error'])
    # ModelCheckpoint
    ckpt_filepath = tempfile.NamedTemporaryFile().name + '.h5'
    ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath = ckpt_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    )
    # CSVLogger
    log = tf.keras.callbacks.CSVLogger(filename=(name_str + '.csv'), append=True)
    callbacks = [early_stopping, ckpt, reduce_lr, log]
    history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=128, validation_split=0.2,
                        callbacks=callbacks)
    model.load_weights(ckpt_filepath)
    y_predicted1 = model.predict(X_test1)
    y_predicted1 = y_predicted1.reshape(y_test1.shape)
    y_predicted2 = model.predict(X_test2)
    y_predicted2 = y_predicted2.reshape(y_test2.shape)
    y_predicted3 = model.predict(X_test3)
    y_predicted3 = y_predicted3.reshape(y_test3.shape)
    coef1 = np.corrcoef(y_test1, y_predicted1)[0,1] ** 2
    coef2 = np.corrcoef(y_test2, y_predicted2)[0,1] ** 2
    coef3 = np.corrcoef(y_test3, y_predicted3)[0,1] ** 2
    eval_1 = model.evaluate(X_test1, y_test1)
    eval_2 = model.evaluate(X_test2, y_test2)
    eval_3 = model.evaluate(X_test3, y_test3)
    result = {'Model': [name_str]*12,
              'Train_site': [site_1]*12,
              'Test_site': [site_1] * 4 + [site_2] * 4 + [site_3] * 4,
              'Metric': ['MSE', 'RMSE', 'MAE', 'coef'] * 3,
              'Value': [eval_1[1], eval_1[2], eval_1[3], coef1,
                        eval_2[1], eval_2[2], eval_2[3], coef2,
                        eval_3[1], eval_3[2], eval_3[3], coef3]}
    df = pd.DataFrame(result)
    return df, history

In [11]:
# Generate evaluation results and correlation coeffcients combined in a dataframe, and history
def double_cross_site(model_name, name_str, lr,
                      site_1, site_2, site_3, X, y):
    # Split the data by sites
    X_1 = X[site==site_1]
    y_1 = y[site==site_1]
    X_2 = X[site==site_2]
    y_2 = y[site==site_2]
    X_3 = X[site==site_3]
    y_3 = y[site==site_3]
    # Split the data into train and test sets:
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_1, y_1, test_size=0.2)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_2, y_2, test_size=0.2)
    X_train3, X_test3, y_train3, y_test3 = train_test_split(X_3, y_3, test_size=0.2)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train1 = np.concatenate([imputer.fit_transform(X_train1[..., ii])[:, :, None] for ii in range(X_train1.shape[-1])], -1)
    X_train2 = np.concatenate([imputer.fit_transform(X_train2[..., ii])[:, :, None] for ii in range(X_train2.shape[-1])], -1)
    X_test1 = np.concatenate([imputer.fit_transform(X_test1[..., ii])[:, :, None] for ii in range(X_test1.shape[-1])], -1)
    X_test2 = np.concatenate([imputer.fit_transform(X_test2[..., ii])[:, :, None] for ii in range(X_test2.shape[-1])], -1)
    X_test3 = np.concatenate([imputer.fit_transform(X_test3[..., ii])[:, :, None] for ii in range(X_test3.shape[-1])], -1)
    # size down evenly
    sample = math.floor(y_test3.shape[0]/2)
    sample1 = resample(X_train1, y_train1, n_samples=sample, replace=False)
    sample2 = resample(X_train2, y_train2, n_samples=sample, replace=False)
    X_train = np.concatenate((sample1[0], sample2[0]), axis=0)
    y_train = np.concatenate((sample1[1], sample2[1]), axis=0)
    # shuffle
    X_train, y_train = shuffle(X_train, y_train)
    X_train, y_train = augment_this(X_train, y_train)
    X_train, y_train = shuffle(X_train, y_train)
    model = model_name(input_shape=(100, 72), n_classes=1, output_activation=None, verbose=True)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['mean_squared_error', 
                           tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                           'mean_absolute_error'])
    # ModelCheckpoint
    ckpt_filepath = tempfile.NamedTemporaryFile().name + '.h5'
    ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath = ckpt_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    )
    # CSVLogger
    log = tf.keras.callbacks.CSVLogger(filename=(name_str + '.csv'), append=True)
    callbacks = [early_stopping, ckpt, reduce_lr, log]
    history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=128, validation_split=0.2,
                        callbacks=callbacks)
    model.load_weights(ckpt_filepath)
    
    y_predicted1 = model.predict(X_test1)
    y_predicted1 = y_predicted1.reshape(y_test1.shape)
    y_predicted2 = model.predict(X_test2)
    y_predicted2 = y_predicted2.reshape(y_test2.shape)
    y_predicted3 = model.predict(X_test3)
    y_predicted3 = y_predicted3.reshape(y_test3.shape)
    coef1 = np.corrcoef(y_test1, y_predicted1)[0,1] ** 2
    coef2 = np.corrcoef(y_test2, y_predicted2)[0,1] ** 2
    coef3 = np.corrcoef(y_test3, y_predicted3)[0,1] ** 2
    eval_1 = model.evaluate(X_test1, y_test1)
    eval_2 = model.evaluate(X_test2, y_test2)
    eval_3 = model.evaluate(X_test3, y_test3)
    result = {'Model': [name_str]*12,
              'Train_site': [f'{site_1}, {site_2}'] * 12,
              'Test_site': [site_1] * 4 + [site_2] * 4 + [site_3] * 4,
              'Metric': ['MSE', 'RMSE', 'MAE', 'coef'] * 3,
              'Value': [eval_1[1], eval_1[2], eval_1[3], coef1,
                        eval_2[1], eval_2[2], eval_2[3], coef2,
                        eval_3[1], eval_3[2], eval_3[3], coef3]}
    df = pd.DataFrame(result)
    return df, history

### cnn_resnet

#### single-cross-site

In [ ]:
df_resnet1, history_resnet1 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 3, 4, X, y)
df_resnet2, history_resnet2 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 3, 0, 4, X, y)
df_resnet3, history_resnet3 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 4, 0, 3, X, y)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 72)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 100, 64)      36928       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 100, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 100, 64)      0           ['batch_normalization[0][0]']

4/4 [==============================] - ETA: 0s - loss: 81.4730 - mean_squared_error: 81.4730 - rmse: 9.0262 - mean_absolute_error: 8.1619   
Epoch 00001: val_loss improved from inf to 23169136.00000, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 5s 619ms/step - loss: 81.4730 - mean_squared_error: 81.4730 - rmse: 9.0262 - mean_absolute_error: 8.1619 - val_loss: 23169136.0000 - val_mean_squared_error: 23169136.0000 - val_rmse: 4813.4331 - val_mean_absolute_error: 4804.0498 - lr: 0.0100
Epoch 2/1000
4/4 [==============================] - ETA: 0s - loss: 13.1788 - mean_squared_error: 13.1788 - rmse: 3.6303 - mean_absolute_error: 2.8078
Epoch 00002: val_loss did not improve from 23169136.00000
4/4 [==============================] - 2s 436ms/step - loss: 13.1788 - mean_squared_error: 13.1788 - rmse: 3.6303 - mean_absolute_error: 2.8078 - val_loss: 232211952.0000 - val_mean_squared_error: 232211952.0000 - val_rmse: 15238

Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 1.4689 - mean_squared_error: 1.4689 - rmse: 1.2120 - mean_absolute_error: 0.9328
Epoch 00017: val_loss improved from 543619.00000 to 285564.34375, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 2s 430ms/step - loss: 1.4689 - mean_squared_error: 1.4689 - rmse: 1.2120 - mean_absolute_error: 0.9328 - val_loss: 285564.3438 - val_mean_squared_error: 285564.3438 - val_rmse: 534.3822 - val_mean_absolute_error: 530.0283 - lr: 0.0100
Epoch 18/1000
4/4 [==============================] - ETA: 0s - loss: 1.3523 - mean_squared_error: 1.3523 - rmse: 1.1629 - mean_absolute_error: 0.9167
Epoch 00018: val_loss improved from 285564.34375 to 158449.54688, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 2s 422ms/step - loss: 1.3523 - mean_squared_error: 1.3523 - rmse: 1.1629 - mean_absolute_error:

Epoch 32/1000
4/4 [==============================] - ETA: 0s - loss: 0.6042 - mean_squared_error: 0.6042 - rmse: 0.7773 - mean_absolute_error: 0.6172
Epoch 00032: val_loss did not improve from 389.09836
4/4 [==============================] - 2s 474ms/step - loss: 0.6042 - mean_squared_error: 0.6042 - rmse: 0.7773 - mean_absolute_error: 0.6172 - val_loss: 415.8896 - val_mean_squared_error: 415.8896 - val_rmse: 20.3934 - val_mean_absolute_error: 19.2816 - lr: 0.0100
Epoch 33/1000
4/4 [==============================] - ETA: 0s - loss: 0.6867 - mean_squared_error: 0.6867 - rmse: 0.8287 - mean_absolute_error: 0.6483
Epoch 00033: val_loss improved from 389.09836 to 92.84183, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 2s 458ms/step - loss: 0.6867 - mean_squared_error: 0.6867 - rmse: 0.8287 - mean_absolute_error: 0.6483 - val_loss: 92.8418 - val_mean_squared_error: 92.8418 - val_rmse: 9.6354 - val_mean_absolute_error: 

Epoch 49/1000
4/4 [==============================] - ETA: 0s - loss: 0.2948 - mean_squared_error: 0.2948 - rmse: 0.5430 - mean_absolute_error: 0.4275
Epoch 00049: val_loss did not improve from 8.86772
4/4 [==============================] - 2s 427ms/step - loss: 0.2948 - mean_squared_error: 0.2948 - rmse: 0.5430 - mean_absolute_error: 0.4275 - val_loss: 13.0964 - val_mean_squared_error: 13.0964 - val_rmse: 3.6189 - val_mean_absolute_error: 2.7594 - lr: 0.0100
Epoch 50/1000
4/4 [==============================] - ETA: 0s - loss: 0.3993 - mean_squared_error: 0.3993 - rmse: 0.6319 - mean_absolute_error: 0.4878
Epoch 00050: val_loss improved from 8.86772 to 8.84559, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmph8yi0czs.h5
4/4 [==============================] - 2s 429ms/step - loss: 0.3993 - mean_squared_error: 0.3993 - rmse: 0.6319 - mean_absolute_error: 0.4878 - val_loss: 8.8456 - val_mean_squared_error: 8.8456 - val_rmse: 2.9742 - val_mean_absolute_error: 2.2086 - lr

...

4/4 [==============================] - ETA: 0s - loss: 80.1182 - mean_squared_error: 80.1182 - rmse: 8.9509 - mean_absolute_error: 8.1217  
Epoch 00001: val_loss improved from inf to 149158752.00000, saving model to /var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmpp7jy2cz8.h5
4/4 [==============================] - 5s 618ms/step - loss: 80.1182 - mean_squared_error: 80.1182 - rmse: 8.9509 - mean_absolute_error: 8.1217 - val_loss: 149158752.0000 - val_mean_squared_error: 149158752.0000 - val_rmse: 12213.0566 - val_mean_absolute_error: 12129.4180 - lr: 0.0100
Epoch 2/1000
4/4 [==============================] - ETA: 0s - loss: 14.4221 - mean_squared_error: 14.4221 - rmse: 3.7976 - mean_absolute_error: 2.9269
Epoch 00002: val_loss did not improve from 149158752.00000
4/4 [==============================] - 2s 388ms/step - loss: 14.4221 - mean_squared_error: 14.4221 - rmse: 3.7976 - mean_absolute_error: 2.9269 - val_loss: 810180096.0000 - val_mean_squared_error: 810180096.0000 - val_rmse: 

4/4 [==============================] - ETA: 0s - loss: 2.3556 - mean_squared_error: 2.3556 - rmse: 1.5348 - mean_absolute_error: 1.1733
Epoch 00016: val_loss did not improve from 10.80274
4/4 [==============================] - 2s 397ms/step - loss: 2.3556 - mean_squared_error: 2.3556 - rmse: 1.5348 - mean_absolute_error: 1.1733 - val_loss: 18.5045 - val_mean_squared_error: 18.5045 - val_rmse: 4.3017 - val_mean_absolute_error: 3.3296 - lr: 0.0100
Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 2.5015 - mean_squared_error: 2.5015 - rmse: 1.5816 - mean_absolute_error: 1.2328
Epoch 00017: val_loss did not improve from 10.80274
4/4 [==============================] - 2s 407ms/step - loss: 2.5015 - mean_squared_error: 2.5015 - rmse: 1.5816 - mean_absolute_error: 1.2328 - val_loss: 41.6713 - val_mean_squared_error: 41.6713 - val_rmse: 6.4553 - val_mean_absolute_error: 5.4112 - lr: 0.0100
Epoch 18/1000
4/4 [==============================] - ETA: 0s - loss: 2.3584 - mean_squ

Epoch 34/1000
4/4 [==============================] - ETA: 0s - loss: 1.1049 - mean_squared_error: 1.1049 - rmse: 1.0511 - mean_absolute_error: 0.8106
Epoch 00034: val_loss did not improve from 9.54471
4/4 [==============================] - 2s 428ms/step - loss: 1.1049 - mean_squared_error: 1.1049 - rmse: 1.0511 - mean_absolute_error: 0.8106 - val_loss: 23.0655 - val_mean_squared_error: 23.0655 - val_rmse: 4.8027 - val_mean_absolute_error: 3.8867 - lr: 0.0100
Epoch 35/1000
4/4 [==============================] - ETA: 0s - loss: 0.9474 - mean_squared_error: 0.9474 - rmse: 0.9733 - mean_absolute_error: 0.7725
Epoch 00035: val_loss did not improve from 9.54471
4/4 [==============================] - 2s 438ms/step - loss: 0.9474 - mean_squared_error: 0.9474 - rmse: 0.9733 - mean_absolute_error: 0.7725 - val_loss: 15.4118 - val_mean_squared_error: 15.4118 - val_rmse: 3.9258 - val_mean_absolute_error: 3.0578 - lr: 0.0100
Epoch 36/1000
4/4 [==============================] - ETA: 0s - loss: 1.110

Epoch 52/1000
4/4 [==============================] - ETA: 0s - loss: 0.5701 - mean_squared_error: 0.5701 - rmse: 0.7551 - mean_absolute_error: 0.5893
Epoch 00052: val_loss did not improve from 8.70640
4/4 [==============================] - 2s 400ms/step - loss: 0.5701 - mean_squared_error: 0.5701 - rmse: 0.7551 - mean_absolute_error: 0.5893 - val_loss: 11.1814 - val_mean_squared_error: 11.1814 - val_rmse: 3.3439 - val_mean_absolute_error: 2.5696 - lr: 0.0100
Epoch 53/1000
4/4 [==============================] - ETA: 0s - loss: 0.4241 - mean_squared_error: 0.4241 - rmse: 0.6512 - mean_absolute_error: 0.5055
Epoch 00053: val_loss did not improve from 8.70640
4/4 [==============================] - 2s 404ms/step - loss: 0.4241 - mean_squared_error: 0.4241 - rmse: 0.6512 - mean_absolute_error: 0.5055 - val_loss: 14.8738 - val_mean_squared_error: 14.8738 - val_rmse: 3.8567 - val_mean_absolute_error: 3.1309 - lr: 0.0100
Epoch 54/1000
4/4 [==============================] - ETA: 0s - loss: 0.523

Epoch 70/1000
4/4 [==============================] - ETA: 0s - loss: 0.5081 - mean_squared_error: 0.5081 - rmse: 0.7128 - mean_absolute_error: 0.5606
Epoch 00070: val_loss did not improve from 7.16676
4/4 [==============================] - 2s 431ms/step - loss: 0.5081 - mean_squared_error: 0.5081 - rmse: 0.7128 - mean_absolute_error: 0.5606 - val_loss: 30.6762 - val_mean_squared_error: 30.6762 - val_rmse: 5.5386 - val_mean_absolute_error: 4.7151 - lr: 0.0100
Epoch 71/1000
4/4 [==============================] - ETA: 0s - loss: 1.0298 - mean_squared_error: 1.0298 - rmse: 1.0148 - mean_absolute_error: 0.8069
Epoch 00071: val_loss did not improve from 7.16676
4/4 [==============================] - 2s 435ms/step - loss: 1.0298 - mean_squared_error: 1.0298 - rmse: 1.0148 - mean_absolute_error: 0.8069 - val_loss: 54.3889 - val_mean_squared_error: 54.3889 - val_rmse: 7.3749 - val_mean_absolute_error: 6.5338 - lr: 0.0100
Epoch 72/1000
4/4 [==============================] - ETA: 0s - loss: 0.874

In [ ]:
df_resnet_1 = (df_resnet1.merge(df_resnet2, how='outer')).merge(df_resnet3, how='outer')

#### double-cross-site

In [ ]:
df_resnet4, hisory_resnet4 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 3, 4, 0, X, y)
df_resnet5, hisory_resnet5 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 4, 3, X, y)
df_resnet6, hisory_resnet6 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 3, 4, X, y)

In [ ]:
df_resnet_2 = (df_resnet4.merge(df_resnet5, how='outer')).merge(df_resnet6, how='outer')

#### cost0

In [ ]:
resnet_cost0 = df_resnet_1[(df_resnet_1['Metric'] == 'MAE') & (df_resnet_1["Test_site"] == 0)]

In [ ]:
resnet_cost0

In [ ]:
resnet_cost0_3 = (resnet_cost0[resnet_cost0['Train_site'] == 3]['Value'].values - 
                  resnet_cost0[resnet_cost0['Train_site'] == 0]['Value'].values)
resnet_cost0_4 = (resnet_cost0[resnet_cost0['Train_site'] == 4]['Value'].values - 
                  resnet_cost0[resnet_cost0['Train_site'] == 0]['Value'].values)

In [ ]:
resnet_cost0_3, resnet_cost0_4

In [ ]:
resnet_cost0_mean = np.mean([resnet_cost0_3, resnet_cost0_4])

In [ ]:
resnet_cost0_0 = df_resnet_2[(df_resnet_2['Metric'] == 'MAE') & (df_resnet_2["Test_site"] == 0)]

In [ ]:
resnet_cost0_0

In [ ]:
resnet_cost0_34 = (resnet_cost0_0[resnet_cost0_0['Train_site'] == '3, 4']['Value'].values - 
                   resnet_cost0[resnet_cost0['Train_site'] == 0]['Value'].values)

In [ ]:
resnet_cost0_34

#### cost3

In [ ]:
resnet_cost3 = df_resnet_1[(df_resnet_1['Metric'] == 'MAE') & (df_resnet_1["Test_site"] == 3)]

In [ ]:
resnet_cost3_0 = (resnet_cost3[resnet_cost3['Train_site'] == 0]['Value'].values - 
                  resnet_cost3[resnet_cost3['Train_site'] == 3]['Value'].values)
resnet_cost3_4 = (resnet_cost3[resnet_cost3['Train_site'] == 4]['Value'].values - 
                  resnet_cost3[resnet_cost3['Train_site'] == 3]['Value'].values)

In [ ]:
resnet_cost3_0, resnet_cost3_4

In [ ]:
resnet_cost3_mean = np.mean([resnet_cost3_0, resnet_cost3_4])

In [ ]:
resnet_cost3_3 = df_resnet_2[(df_resnet_2['Metric'] == 'MAE') & (df_resnet_2["Test_site"] == 3)]

In [ ]:
resnet_cost3_3

In [ ]:
resnet_cost3_04 = (resnet_cost3_3[resnet_cost3_3['Train_site'] == '0, 4']['Value'].values - 
                   resnet_cost3[resnet_cost3['Train_site'] == 3]['Value'].values)

In [ ]:
resnet_cost3_04

#### cost4

In [ ]:
resnet_cost4 = df_resnet_1[(df_resnet_1['Metric'] == 'MAE') & (df_resnet_1["Test_site"] == 4)]

In [ ]:
resnet_cost4_0 = (resnet_cost4[resnet_cost4['Train_site'] == 0]['Value'].values - 
                  resnet_cost4[resnet_cost4['Train_site'] == 3]['Value'].values)
resnet_cost4_3 = (resnet_cost4[resnet_cost4['Train_site'] == 4]['Value'].values - 
                  resnet_cost4[resnet_cost4['Train_site'] == 3]['Value'].values)

In [ ]:
resnet_cost4_mean = np.mean([resnet_cost4_0, resnet_cost4_3])

In [ ]:
resnet_cost3_3 = df_resnet_2[(df_resnet_2['Metric'] == 'MAE') & (df_resnet_2["Test_site"] == 3)]